In [ ]:
###############################################################################
import aliases
from everest.utilities import textutils
# tuple(textutils.find_all('_repr_pretty_', path='../everest/ptolemaic'))
# textutils.replace_all('Schematic', 'Compound', path='../everest')

In [ ]:
from everest.ptolemaic.algebra import *
from everest.ptolemaic.enumm import Enumm

In [ ]:
class Foo(Realm):

    class symbols(metaclass=Enumm):
        a: None
        b: None
        c: None
        d: None
        e: None
        f: None

    class algebra(mroclass):

        zero = operator()
        neg = operator(1, invertible=True)
        add = operator(Ennary, associative=False, commutative=False)
        mul = operator(
            Ennary,
            associative=False, commutative=False, distributive=pathget('.add')
            )

foo = Foo()
alg = foo.algebra

In [ ]:
form = alg.mul(
    foo.symbols.a,
    alg.add(foo.symbols.b, foo.symbols.c),
    alg.add(foo.symbols.d, foo.symbols.e),
    foo.symbols.e,
    )
# a * (b + c) * (d + e) * f == a*b*d*f + a*c*d*f + a*b*e*f + a*c*e*f
#                           == (a*b + a*c) * (d + e) * f

In [ ]:
form

In [ ]:
form.canonise()

In [ ]:
from collections import deque as _deque
import itertools as _itertools


def distribute(factors, under, /):
    chunks = _deque()
    buff = _deque()
    for factor in factors:
        if factor.operator is under:
            chunks.append(tuple((*buff, term) for term in factor.arguments))
            buff.clear()
        else:
            buff.append(factor)
    if chunks:
        chunks.append(tuple((*term, *buff) for term in chunks.pop()))
        return tuple(map(tuple, map(
            _itertools.chain.from_iterable, _itertools.product(*chunks)
            )))
    return (tuple(buff),)

In [ ]:
distributed = tuple(distribute(form.arguments, foo.add))

In [ ]:
distributed

In [ ]:
for item in distribute(form.arguments, foo.add):
    display(tuple(item))

In [ ]:
distributed[0]

In [ ]:
distributed[1]

In [ ]:
a * (b + c) * (d + e)

In [ ]:
a * (b + c) * d == a*d * (b + c) == a*d*b + a*d*c

In [ ]:
a, b, c, d, e, f = range(2, 8)
g = a * b
h = c + d
i = e + f
assert a * b * (c + d) * (e + f) \
    == g * h * i \
    == (a*b*c + a*b*d) * (e + f) \
    == a*b*c*e + a*b*c*f + a*b*d*e + a*b*d*f

In [ ]:
assert a * b * (c + d) * e * (f + g) * h \
    == (a*b*c + a*b*d) * (e*f + e*g) * h \
    == (a*b*c + a*b*d) * (e*f*h + e*g*h)

In [ ]:
def simple_distribute(distr, prefactors, expression, postfactors, /):
    return distr(*((*prefactors, arg, *postfactors) for arg in expression.args))

def distribute(distr, expression, /):
    args = expression.args
    isdistrs = tuple(isinstance(arg, distr) for arg in args)
    if not any(isdistrs):
        return expression
    # outterms = []
    # while True:
    buff = []
    newargs = []
    for arg, isdistr in zip(args, isdistrs):
        if isdistr:
            newargs.append(simple_distribute(distr, buff, arg, ()))
            buff.clear()
        else:
            buff.append(arg)
    if buff:
        newargs[-1] = tuple((*term, *buff) for term in newargs[-1].args)
    # if len(newargs) == 1:
    #     return newargs[0]
    return type(expression)(*newargs)

out = distribute(
    Add,
    Mul('a', 'b', Add('c', 'd'), 'e', Add('f', 'g'), 'h'),
    )
display(out)

In [ ]:
class Distributor:

    __slots__ = ('mul', 'add')

    def __init__(self, mul, add, /):
        self.mul, self.add = mul, add

    class Op:
        __slots__ = ('args',)
        def __init__(self, *args):
            self.args = args
        def __repr__(self, /):
            return f"{self.__class__.__name__}{self.args}"

    class Add(Op):
        ...

    class Mul(Op):
        ...

    class Other:
        def __init__(self, arg, /):
            self.arg = arg
        def __repr__(self, /):
            return f"{self.__class__.__name__}({self.arg})"

    def preprocess(self, arg, /):
        if isinstance(arg, self.mul):
            return self(*arg.args)
            return self.Mul(*arg.args)
        if isinstance(arg, self.add):
            return self.Add(*arg.args)
        return arg

    def combine(self, arg0, arg1):
        

    def __call__(self, /, *args):
        return self.Add(*map(self.preprocess, args))

In [ ]:
class Base:
    __slots__ = ('args',)
    def __init__(self, *args):
        self.args = args
    def __repr__(self, /):
        return f"{self.__class__.__name__}{self.args}"
class MyMul(Base):
    ...
class MyAdd(Base):
    ...
distributor = Distributor(MyMul, MyAdd)
distributor('a', 'b', MyAdd('c', 'd'), 'e', MyMul('f', 'g'))

In [ ]:
distribute(
    Add,
    Mul('a', 'b', Add('c', 'd'), 'e', Add('f', 'g'), 'h'),
    )

In [ ]:
Distribute(1)

In [ ]:
distribute(
    Add,
    Mul('a', 'b', Add('c')),
    )

In [ ]:
distribute(Add, out)

In [ ]:
assert a * b * (c + d) * e * (f + g) * h \
    == (a*b*c + a*b*d) * (e*f + e*g) * h \
    == (a*b*c + a*b*d) * (e*f*h + e*g*h) \
    == a*b*c*e*f*h + a*b*c*e*g*h + a*b*d*e*f*h + a*b*d*e*g*h

In [ ]:
distribute_over(('a', 'b'), ('c', 'd'))

In [ ]:

from everest.ptolemaic.brace import Brace

In [ ]:
class MyAlgebra(metaclass=Algebra):

    class Brace(Brace):
        ...

In [ ]:
MyAlgebra.Brace.Power

In [ ]:

from everest.ptolemaic.system import System

In [ ]:
class A(metaclass=System):
    a: POS[int]
    foo = str
class B(A):
    a: POS['.foo']
class C(B):
    foo = bool
C.__signature__

In [ ]:
B.__fields__

In [ ]:
class A(metaclass=Algebra):
    @classmethod
    def _algconvert_(cls, arg, /):
        return arg
    class Identity(mroclass('.__Nullary__'), metaclass=System):
        ...
    # class MyOp(mroclass('.__Ennary__')):
    #     identity = pathget('..IDENTITY', live=True)
    #     @classmethod
    #     def _construct_(cls, params, /):
    #         return params[0]

In [ ]:
class Natural(int):

    def __init__(self, value, /):
        if value < 0:
            raise ValueError(value)
        super().__init__()

    def __repr__(self, /):
        return f"Natural({super().__repr__()})"

    def __sub__(self, other, /):
        return type(self)(super().__sub__(other))

In [ ]:
Natural(1) - 1

In [ ]:
from everest.ptolemaic.essence import Essence as _Essence
from everest.ptolemaic.algebra import Algebra as _Algebra
from everest.ptolemaic.sett import Sett as _Sett
from everest.ptolemaic.brace import Brace as _Brace
from everest.ptolemaic.ptolemaic import Ptolemaic as _Ptolemaic


class Objj(metaclass=_Algebra):


    @classmethod
    def __class_init__(cls, /):
        super().__class_init__()
        cls.Sett.Element = cls.Base
        if cls is __class__:
            cls.Base.register(_Ptolemaic)


    class Base(mroclass):

        ...


    class Sett(mroclass(_Sett)):

        ...


    class Brace(mroclass(_Brace)):

        ...

In [ ]:
[] in _Sett.UNIVERSE

In [ ]:
Objj in Objj.Sett.UNIVERSE

In [ ]:
# textutils.replace_all('Schematic', 'Compound', path='../everest')

In [ ]:
from everest.algebraic import sett, mapp, chora
from everest.ptolemaic.system import System

In [ ]:
class MyChora(chora.Choret, metaclass=System):

    lower: POS[int]
    upper: POS[int] = None

    @classmethod
    def __parameterise__(cls, /, *args, **kwargs):
        params = super().__parameterise__(*args, **kwargs)
        if params.upper is None:
            params.upper = params.lower
            params.lower = 0
        return params

    def _incise_handle_int_(self, incisor: int, /):
        return chora.IncisionStyle.RETRIEVE(incisor)

    def _incise_handle_slice_(self, incisor: slice, /):
        start, stop, step = incisor.start, incisor.stop, incisor.step
        if step is not None:
            raise ValueError(incisor)
        return chora.IncisionStyle.SLYCE((start, stop))

    def _incise_handle_tuple(self, incisor: tuple, /):
        return chora.IncisionStyle.SLYCE(incisor)

    def _incise_handle_ellipsis_(self, incisor: type(Ellipsis), /):
        return chora.IncisionStyle.TRIVIAL(incisor)

    def _incise_retrieve_int_(self, incisor: int, /):
        return self.lower + incisor if incisor >= 0 else self.upper + incisor

    def _incise_slyce_tuple_(self, lower, upper, /):
        slower, supper = self.params
        return self.__ptolemaic_class__(
            slower if lower is None else \
                (slower if lower >= 0 else supper) + lower,
            supper if upper is None else \
                (slower if upper >= 0 else supper) + upper,
            )

    def __iter__(self, /):
        return iter(range(*self.params))

    def _contains_(self, other, /):
        return self.lower <= other <= self.upper


achora = MyChora(3, 9)
bchora = MyChora(5, 15)
cchora = bchora @ achora
cchora[3:]

In [ ]:
class MyClass(chora.Chorelle, metaclass=System):

    chora: chora.Chora

    def _incise_slyce_(self, incisor, /):
        return self.__ptolemaic_class__(super()._incise_slyce_(incisor))


dchora = MyClass(cchora)
assert dchora[...] is dchora
dchora[3:]

In [ ]:
echora = chora.MultiChora(achora, bchora)
display(echora[0, 2:])
display(echora[0, 2:][0])
(3, 7) in echora